In [ ]:
import pandas as pd
import tempfile
import re
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import drive
import csv
from tqdm import tqdm_notebook as tqdm

In [ ]:
pip install ujson

     |████████████████████████████████| 194kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson


In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Pre-processing

In [ ]:
reddit_data_paths_proED = ['/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proED_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proED_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edFood_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edFood_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proanaweightloss_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proanaweightloss_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspo_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspo_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspocommunity_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspocommunity_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_trueThinspo_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_trueThinspo_coms.json',
                    ]

reddit_data_paths_proRec = ['/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edAnonymous_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edAnonymous_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bingeEatingDisorder_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bingeEatingDisorder_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_eating_disorders_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_eating_disorders_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edsupport_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edsupport_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bulimia_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bulimia_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_myProAna_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_myProAna_coms.json',
                    ]

In [ ]:
def json_to_df(path, type):
  if ( type == "post"):
    return pd.read_json(path)[["author", "id","title", "selftext", "num_comments", "score","created_utc", "subreddit"]]
  elif ( type == "comment"):
    return pd.read_json(path)[["id", "author", "created_utc", "link_id", "parent_id", "score", "body", "subreddit"]]

In [ ]:
def merge_dataframes(reddit_data_paths):
  df_all_posts    = json_to_df(reddit_data_paths[0], "post")
  df_all_comments = json_to_df(reddit_data_paths[1], "comment")

  for i in tqdm(range(2, len(reddit_data_paths), 2)):
    #read
    df_posts    = json_to_df(reddit_data_paths[i], "post")
    df_comments = json_to_df(reddit_data_paths[i+1], "comment")
    #append
    df_all_posts.append(df_posts)
    df_all_comments.append(df_comments)

  #set the id as index
  posts = df_all_posts.set_index("id")
  comts = df_all_comments.set_index("id")

  #drop deleted, removed, empty bodies
  return posts[posts.selftext != "[removed]"][posts.selftext != "[deleted]"][posts.selftext != ""][posts.selftext != "None"], comts[comts.body != "[removed]"][comts.body != "[deleted]"][comts.body != ""][comts.body != "None"]

In [ ]:
# it changes the df_posts given in the parameters
# like poiter...
def fill_author(df_posts):
  dic = df_posts[df_posts.author=="[deleted]"].to_dict()['author']
  df = df_posts
  for key, value in tqdm(dic.items(), total=len(dic)):
    s = str(tempfile.NamedTemporaryFile().name.split('/')[2][3:])
    df.loc[key, "author"] = s

In [ ]:
ed_posts,  ed_comments  = merge_dataframes(reddit_data_paths_proED)
rec_posts, rec_comments = merge_dataframes(reddit_data_paths_proRec)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
fill_author(rec_posts)
fill_author(rec_comments)
fill_author(ed_posts)
fill_author(ed_comments)

In [ ]:
def initialize(df_posts , df_comments, author_posts_comments, author_comments):

  author_comments = {}
  author_posts_comments = {}

  #from posts
  for ind, row in tqdm(df_posts.iterrows(), total = len(df_posts)):
    author_posts_comments[row["author"]] = []

  #from comments
  for ind, row in tqdm(df_comments.iterrows(), total = len(df_comments)):
    if (row["author"] in author_posts_comments == 'FALSE'):
      author_posts_comments[row["author"]] = []
    else:
      author_comments[row["author"]] = []


  return author_posts_comments, author_comments

In [ ]:
#asigning to each author his things
#each post has many comments
#each author has many comments/posts
def asign_child_to_parent(df_posts, df_comments, author_posts_comments, author_comments):

  #asign post to author
  for ind, row in tqdm(df_posts.iterrows(), total=len(df_posts)):
    author_posts_comments[row["author"]].append(ind)


  for ind, row in tqdm(df_comments.iterrows(), total=len(df_comments)):
    #asign comment to author
    if (row["author"]in author_posts_comments):
      author_posts_comments[row["author"]].append(ind)
    else:
      author_comments[row["author"]].append(ind)


  return author_posts_comments, author_comments

In [ ]:
def ids_to_texts(df_posts, df_comments, author_posts_comments):
  for key, value in tqdm(author_posts_comments.items(), total=len(author_posts_comments)):
    for ind, i in enumerate(value):
      if(i in df_posts.index):
        if(df_posts.loc[i].title is None or df_posts.loc[i].selftext is None or df_posts.loc[i].title == '' or df_posts.loc[i].selftext == ''):
          value[ind] = 'None'
        else:
          value[ind] = df_posts.loc[i].title + " " + df_posts.loc[i].selftext

      elif (i in df_comments.index):
        if(df_comments.loc[i].body is None or df_comments.loc[i].body ==  ''):
          value[ind] = 'None'
        else:
          value[ind] = df_comments.loc[i].body

  return author_posts_comments

In [ ]:
def ids_to_texts_comments(df_posts, df_comments, author_comments):
  for key, value in tqdm(author_comments.items(), total=len(author_comments)):
    for ind, i in enumerate(value):
      if (i in df_comments.index):
        if(df_comments.loc[i].body is None or df_comments.loc[i].body == ''):
          value[ind] = 'None'
        else:
          value[ind] = df_comments.loc[i].body
  return author_comments

In [ ]:
def list_to_text(l):
  str = " ".join(l)
  return str

In [ ]:
def final_dicts(author_posts_comments, author_comments):
  for key, value in tqdm(author_posts_comments.items(), total=len(author_posts_comments)):
    author_posts_comments[key] = list_to_text(value)
  for key, value in tqdm(author_comments.items(), total=len(author_comments)):
    author_comments[key] = list_to_text(value)

  return author_posts_comments, author_comments

In [ ]:
def dicts_to_dataframes(author_posts_comments, author_comments):
  df_users_posts = pd.DataFrame(author_posts_comments.items(), columns=['users', 'content'])
  df_users_comments = pd.DataFrame(author_comments.items(), columns=['users', 'content'])
  return df_users_posts, df_users_comments

###execute

In [ ]:
author_posts_comments_rec, author_comments_rec = initialize(rec_posts , rec_comments, {}, {})
author_posts_comments_rec, author_comments_rec = asign_child_to_parent(rec_posts, rec_comments, author_posts_comments_rec, author_comments_rec)

In [ ]:
author_posts_comments_ed, author_comments_ed = initialize(ed_posts , ed_comments, {}, {})
author_posts_comments_ed, author_comments_ed = asign_child_to_parent(ed_posts, ed_comments, author_posts_comments_ed, author_comments_ed)

In [ ]:
author_posts_comments_rec = ids_to_texts(rec_posts, rec_comments, author_posts_comments_rec)
author_comments_rec = ids_to_texts_comments(rec_posts, rec_comments, author_comments_rec)

In [ ]:
author_posts_comments_ed = ids_to_texts(ed_posts, ed_comments, author_posts_comments_ed)
author_comments_ed = ids_to_texts_comments(ed_posts, ed_comments, author_comments_ed)

In [ ]:
author_posts_comments_rec, author_comments_rec = final_dicts(author_posts_comments_rec, author_comments_rec)

In [ ]:
author_posts_comments_ed, author_comments_ed = final_dicts(author_posts_comments_ed, author_comments_ed)

In [ ]:
df_users_posts_rec, df_users_comments_rec = dicts_to_dataframes(author_posts_comments_rec, author_comments_rec)

In [ ]:
df_users_posts_ed, df_users_comments_ed = dicts_to_dataframes(author_posts_comments_ed, author_comments_ed)

In [ ]:
df_ed = pd.DataFrame()

In [ ]:
df_users_comments_ed = df_users_comments_ed[df_users_comments_ed.content != '']
df_users_comments_ed.head(10)

,users,content
0,Klairvoyant,[]
1,Knyburg,[]
2,FascistAsparagus,[]
3,Alexa-the-hexa,[]
4,byebyeb1rd,[**Breakfast:** Greek Yogurt-100 cal\n\n**Lunc...
5,ifuckpineapples,[]
6,Someone_Who_Isnt_You,[]
7,explodingeyeballs,[Dude I'm the same size too!! Yeah I know righ...
8,WithyWillow,[]
9,PurgingPisces,[Congrats! Hopefully to be in your shoes soon!...


In [ ]:
df_users_posts_ed = df_users_posts_ed[df_users_posts_ed.content != '']
df_users_posts_ed.head(10)

,users,content
0,Knyburg,"[None, None, None, None, None, ED Tip: When ru..."
1,Klairvoyant,[The proED Diet/Workout Plan \n1. No simple ca...
2,AutoModerator,"[Daily Food Diary! August 09, 2015 This is a d..."
3,lunaticsloverspoets,[Baby's First EC Stack You guys... I don't kno...
4,HowToBeAsian25,[Need some help determining goals I'm a very v...
5,ImOkWhyDoYouAsk,"[None, Mid-cycle bleeding, anyone else?? I've ..."
6,Noroeste,"[None, None, None, None, None, Low-cal options..."
7,goddamnroommate,[Nausea with EC stacks? Anyone else experience...
8,notyourtoy,[Just another post about how wonderful you peo...
9,THORFINN_THE_DANE,[sup fellow hungry people I've run out of mone...


In [ ]:
df_users_posts_rec = df_users_posts_rec[df_users_posts_rec.content != '']
df_users_posts_rec.head(10)

,users,content
0,thingsarestranger,Something positive you learned/experienced fro...
1,AnnaGreen3,Can we do something? Is there something we can...
2,taikutsuu,Please recognize that there is a realistic cha...
3,macespacee,Depression medication and appetite// weight lo...
4,raspbunni,reached a new hw :):):):) i stuffed myself lik...
5,probablyfuckedurdad,I wish people were more concerned about me. Is...
6,citrus_juuiice,"I ate normal for two days I feel so gross, I a..."
7,lovedieshere,Overeater finally became an undereater... So I...
8,illmurderyoudickhead,BMI of 21.29. Ashamed of myself. Title says it...
9,PoemOfLifeItself,you know that feeling when... you shit before ...


In [ ]:
df_users_comments_rec = df_users_comments_rec[df_users_comments_rec.content != '']
df_users_comments_rec.head(10)

,users,content
0,toritxtornado,"iToldAnotherLieToday i’m in recovery now, but ..."
2,sabrinaxspellman,@sabrinaxspellman \nI’ve never used peach befo...
13,whitebeaks,"\&gt;my skinny friends gained some \nFuck, it..."
15,wristsPlz,"Tried to add you but it didn't work, are there..."
18,Sunshine-Fox,"I don't do this, but growing up my anorexic mo..."
20,IcyPalpitation,@Icy_IceBaby. Add me please :) Bread. Especial...
21,Bot_Metric,20.0 lbs ≈ 9.1 kilograms ^(1 pound ≈ 0.45kg)\n...
26,ohdarlingyourefucked,I’m here! I’m so upset about this. Currently u...
27,cocaineinmycereal,Feelin this 👌😂 Absolutely! I’ve never been to ...
30,Canibalope,is it too late? It honestly blows my mind that...


### save


In [ ]:
df_users_comments_ed[df_users_comments_ed.content != ''].to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_posts_ed.csv')
df_users_posts_ed.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_comments_ed.csv')
df_users_posts_rec.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_comments_rec.csv')
df_users_comments_rec[df_users_comments_rec.content != ''].to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_posts_rec.csv')

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import lxml.html
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Algorithm (LDA)

In [ ]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
pip install emoji

In [ ]:
from emoji import UNICODE_EMOJI

In [ ]:
data1 = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_posts_ed.csv')
data2 = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_comments_ed.csv')
df_ed = data1.append(data2, ignore_index=True)
data3 = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_comments_rec.csv')
data4 = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_8/df_users_posts_rec.csv')
df_rec = data3.append(data4, ignore_index=True)d

In [ ]:
len(df_rec)

7853

In [ ]:
len(df_ed)

14681

## drop nan

In [ ]:
df_ed[df_ed.isna().any(axis=1)]


,Unnamed: 0,users,content


In [ ]:
df_rec[df_rec.isna().any(axis=1)]

,Unnamed: 0,users,content


In [ ]:
df_ed = df_ed.dropna()

In [ ]:
df_ed[df_ed.isna().any(axis=1)]

,Unnamed: 0,users,content


In [ ]:
df_ed = df_ed[df_ed.content != '[]']
df_rec = df_rec.dropna()

In [ ]:
df_rec = df_rec[df_rec.content != '[]']

In [ ]:
df_ed.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_ed.csv')
df_rec.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_rec.csv')

In [ ]:
len(df_ed)

11588

In [ ]:
len(df_rec)

7853

In [ ]:
df_ed = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_ed.csv')

In [ ]:
df_rec= pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_rec.csv')

## end drop nan

In [ ]:
def valid_text(text):
  if(text is None or text == '' or text == np.nan):
    return False
  else:
    return True

In [ ]:
def remove_tags(text):
  if (valid_text(text)):
    if (lxml.html.fromstring(text).find('.//*') is not None):
      return re.sub('<[^>]*>', '', text)
    else:
      return text

In [ ]:
def is_emoji(s):
  for emoji in UNICODE_EMOJI:
    if(emoji in str(s)):
      return True
    else:
      return False

In [ ]:
def deEmojify(s):
  if(is_emoji(s)):
    return s.encode('ascii', 'ignore').decode('ascii')
  else:
    return s

In [ ]:
#Split words
def splitting(text):
  tokens = ""
  try:
    tokens = word_tokenize(text)
  except:
    if(text == np.nan):
      print("nan")
    print("not nan")
  return tokens

In [ ]:
def remove_Nones(words):
  text = []
  for w in words:
    if (w != 'None'):
      text.append(w)
  return text

In [ ]:
def delinks(words):
  #text = [re.sub(r'''(?i)\b((?:(https|http)?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', w, flags=re.MULTILINE) for w in words]
  text = [re.sub(r"http\S+", "", w) for w in words]
  return text

In [ ]:
'''words = ['jfhksfhkd http://www.datacamp.com/community/tutorials/lda2vec-topic-model', 'hfqhsdhfhdg', 'https://www.datacamp.com/community/tutorials/lda2vec-topic-model']
print(delinks(words))'''


"words = ['jfhksfhkd http://www.datacamp.com/community/tutorials/lda2vec-topic-model', 'hfqhsdhfhdg', 'https://www.datacamp.com/community/tutorials/lda2vec-topic-model']\nprint(delinks(words))"

In [ ]:
def remove_numbers(words):
  output = [re.sub(r'\d+', '', w) for w in words]
  return output

In [ ]:
def depunct(tokens):
  # remove all tokens that are not alphabetic
  words = [word for word in tokens if word.isalpha()]
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in words]
  return stripped

In [ ]:
def de_stopwords(words):
  stop_words = stopwords.words('english')
  stop_words.extend(['http', 'lw', 'gw', 'ugw', 'f', 'u'])
  l = [w for w in words if not w in stop_words]
  return l

In [ ]:
def tolowercase(words):
  text = [w.lower() for w in words]
  return text

In [ ]:
def stemming(tokens):
  porter = PorterSt  emmer()
  stemmed = [porter.stem(word) for word in tokens]
  return stemmed

In [ ]:
def load_texts(text):
  inp = deEmojify(text)
  return splitting(inp)

In [ ]:
def pre_processing(l):
  no_none = remove_Nones(l)
  no_link = delinks(no_none)
  no_numb = remove_numbers(no_link)
  return stemming(de_stopwords(tolowercase(depunct(no_numb))))

In [ ]:
def dataframe_preprocessing(df):
  contents = []
  users = []
  for ind, row in tqdm(df.iterrows(), total=len(df)):
    users.append(row["users"])
    #l = pre_processing(load_texts(row["content"]))
    txt = load_texts(row["content"])
    contents.append(pre_processing(txt))
  return users, contents

## execute

In [ ]:
u_ed, c_ed = dataframe_preprocessing(df_ed)

In [ ]:
u_rec, c_rec = dataframe_preprocessing(df_rec)

#Bag of words

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [ ]:
dictionary_ed = gensim.corpora.Dictionary(c_ed)

In [ ]:
dictionary_rec = gensim.corpora.Dictionary(c_rec)

In [ ]:
dictionary_ed.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
dictionary_rec.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus_ed = [dictionary_ed.doc2bow(doc) for doc in c_ed]

In [ ]:
bow_corpus_rec = [dictionary_rec.doc2bow(doc) for doc in c_rec]

In [ ]:
bow_corpus_rec

#LDA using bag of words

In [ ]:
lda_model_ed = gensim.models.ldamodel.LdaModel(bow_corpus_ed, num_topics=15, id2word=dictionary_ed, random_state=100, update_every=1, chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [ ]:
lda_model_rec = gensim.models.ldamodel.LdaModel(bow_corpus_rec, num_topics=9, id2word=dictionary_rec, random_state=100, update_every=1, chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [ ]:
for idx, topic in lda_model_ed.print_topics(-1):
  print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.072*"cal" + 0.046*"coffe" + 0.040*"g" + 0.037*"cup" + 0.030*"today" + 0.027*"dinner" + 0.023*"veggi" + 0.022*"salad" + 0.020*"soup" + 0.017*"black"
Topic: 1 
Words: 0.048*"watch" + 0.040*"show" + 0.035*"thinspo" + 0.033*"favorit" + 0.026*"measur" + 0.021*"emot" + 0.021*"bag" + 0.021*"write" + 0.020*"lurk" + 0.017*"iron"
Topic: 2 
Words: 0.052*"like" + 0.030*"feel" + 0.029*"look" + 0.023*"realli" + 0.020*"want" + 0.018*"know" + 0.017*"get" + 0.017*"much" + 0.016*"love" + 0.015*"thank"
Topic: 3 
Words: 0.020*"enough" + 0.016*"last" + 0.016*"come" + 0.014*"guess" + 0.013*"worri" + 0.012*"alreadi" + 0.011*"told" + 0.010*"hit" + 0.010*"sub" + 0.010*"final"
Topic: 4 
Words: 0.027*"skinni" + 0.018*"muscl" + 0.017*"believ" + 0.016*"period" + 0.015*"pictur" + 0.011*"hunger" + 0.011*"suggest" + 0.011*"fact" + 0.011*"motiv" + 0.010*"lax"
Topic: 5 
Words: 0.070*"calori" + 0.038*"protein" + 0.032*"tea" + 0.026*"sugar" + 0.024*"drink" + 0.023*"chees" + 0.022*"water" + 0.021*"food"

In [ ]:
for idx, topic in lda_model_rec.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

#LDA visualization

In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
import pyLDAvis


##Bag of words

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts, users):
    # Init output
    topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topics_df = topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    redditors = pd.Series(users)
    topics_df = pd.concat([topics_df, contents, redditors], axis=1)
    return(topics_df)

In [ ]:
df_topic_keywords_ed = format_topics_sentences(lda_model_ed, bow_corpus_ed, c_ed, u_ed)

# Format
df_dominant_topic_ed = df_topic_keywords_ed.reset_index()
df_dominant_topic_ed.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Redditors']

In [ ]:
df_dominant_topic_ed.head(10)



In [ ]:
df_dominant_topic_ed.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/topics/df_topics_ed.csv')

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_ed, bow_corpus_ed, dictionary=lda_model_ed.id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.334392 -0.016432       1        1  32.386707
2     -0.324862 -0.079002       2        1  20.488258
6     -0.266721 -0.074914       3        1   8.424837
3      0.158647 -0.294112       4        1   7.871070
7      0.158890 -0.262677       5        1   4.319838
5     -0.070640  0.017973       6        1   4.016851
10     0.000081  0.122575       7        1   3.820478
4      0.146352 -0.140788       8        1   3.782398
0      0.086474  0.142661       9        1   3.362924
14    -0.104766  0.077291      10        1   2.759639
12     0.117361  0.105174      11        1   2.451657
9      0.089377  0.144155      12        1   2.387199
1      0.128579  0.088978      13        1   1.742088
11     0.106370  0.091285      14        1   1.237035
8      0.109250  0.077835      15        1   0.949021, topic_info=     Category          Freq      Term         Total  loglift  logprob
29    Default  40301.000000    calori  40301.000000  30.0000  30.0000
101   Default  93387.000000      like  93387.000000  29.0000  29.0000
51    Default  78033.000000       eat  78033.000000  28.0000  28.0000
73    Default  58309.000000       get  58309.000000  27.0000  27.0000
176   Default  21199.000000      take  21199.000000  26.0000  26.0000
69    Default  37195.000000      food  37195.000000  25.0000  25.0000
343   Default  15056.000000        lb  15056.000000  24.0000  24.0000
107   Default  35571.000000      look  35571.000000  23.0000  23.0000
42    Default  55690.000000       day  55690.000000  22.0000  22.0000
177   Default  20351.000000     thank  20351.000000  21.0000  21.0000
63    Default  66322.000000      feel  66322.000000  20.0000  20.0000
383   Default  27272.000000     peopl  27272.000000  19.0000  19.0000
561   Default  44383.000000    weight  44383.000000  18.0000  18.0000
28    Default  14671.000000       cal  14671.000000  17.0000  17.0000
454   Default  21845.000000       use  21845.000000  16.0000  16.0000
183   Default  14592.000000     today  14592.000000  15.0000  15.0000
530   Default  11813.000000      post  11813.000000  14.0000  14.0000
94    Default  42518.000000      know  42518.000000  13.0000  13.0000
127   Default  35415.000000       one  35415.000000  12.0000  12.0000
310   Default  11025.000000      goal  11025.000000  11.0000  11.0000
203   Default  23809.000000      work  23809.000000  10.0000  10.0000
274   Default  16119.000000     drink  16119.000000   9.0000   9.0000
179   Default  39208.000000     think  39208.000000   8.0000   8.0000
325   Default  25242.000000      help  25242.000000   7.0000   7.0000
170   Default  23156.000000     start  23156.000000   6.0000   6.0000
197   Default  15422.000000     water  15422.000000   5.0000   5.0000
31    Default  11987.000000     coffe  11987.000000   4.0000   4.0000
14    Default  29245.000000      bing  29245.000000   3.0000   3.0000
563   Default  29442.000000     would  29442.000000   2.0000   2.0000
196   Default  44394.000000      want  44394.000000   1.0000   1.0000
...       ...           ...       ...           ...      ...      ...
2028  Topic15   1266.415039     insid   1267.750732   4.6564  -3.8205
3678  Topic15   1166.323975      sign   1167.659668   4.6564  -3.9029
2302  Topic15   1120.287842        ex   1121.623535   4.6563  -3.9432
2519  Topic15   1038.022949      dish   1039.358643   4.6562  -4.0194
3413  Topic15    983.467529   younger    984.803284   4.6561  -4.0734
4028  Topic15    869.492310    suicid    870.828064   4.6560  -4.1966
2426  Topic15    827.239685   luckili    828.575439   4.6559  -4.2464
2626  Topic15    825.299377    nausea    826.635132   4.6559  -4.2487
2628  Topic15    814.017822     older    815.353577   4.6559  -4.2625
3243  Topic15    792.623291      till    793.959045   4.6558  -4.2891
2889  Topic15    708.617554    genuin    709.953308   4.6556  -4.4012
2922  Topic15    686.268555  

## pro recovery

In [ ]:
df_topic_keywords_rec = format_topics_sentences(lda_model_rec, bow_corpus_rec, c_rec, u_rec)

# Format
df_dominant_topic_rec = df_topic_keywords_ed.reset_index()
df_dominant_topic_rec.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Redditors']

In [ ]:
df_dominant_topic_rec.head(10)

In [ ]:
df_dominant_topic_rec.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/topics/df_topics_rec.csv')

In [ ]:
pyLDAvis.enable_notebook()
vis_rec = pyLDAvis.gensim.prepare(lda_model_rec, bow_corpus_rec, dictionary=lda_model_rec.id2word)
vis_rec

coherence scores:

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_ed, texts=c_ed, dictionary=dictionary_ed, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_ed, texts=c_ed, dictionary=dictionary_ed, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  -0.8953079998227452


rec

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_rec, texts=c_rec, dictionary=dictionary_rec, coherence='c_v')
coherence_lda_rec = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda_rec)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_rec, texts=c_rec, dictionary=dictionary_rec, coherence='u_mass')
coherence_lda_rec = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda_rec)